In [ ]:
# 43679 -- Interactive Visualization
# 2025 - 2026
# 2nd semester
# Lab 1 - EDA (guided)
# ver 1.0 - 2026-02-20 Initial version
# ver 1.1 - 2026-02-23  Added more comments and explanations
# ver 1.2 - 2026-02-24  Added code for additional visualizations

# Lab 01<br>Task 1: Exploratory Data Analysis with Pandas & Seaborn

This task serves two purposes. It introduces you to some of the basic tools to start understanding datasets and shows you why descriptive statistics may not be enough to understand the nature of a dataset.

Also, this task also walks you through some basic visualizations of the datasets to show how the type of visualization matters when trying to understand the data.

Additionally, this simple first task also serves the purpose of getting you acquainted with Jupyter notebooks.

**Dataset:** `datasaurus.csv`

---

### Objectives

By the end of this task you will be able to:
- Use `pandas` to inspect a dataset's structure, types, and summary statistics
- Apply grouped aggregations to compare subsets of data
- Use `seaborn` to produce scatter plots that reveal structure invisible to statistics
- Articulate *why* visualisation is an essential — not optional — step in data analysis

---

### Context

The **Datasaurus Dozen** is a collection of 13 small datasets created by Matejka & Fitzmaurice (2017) to demonstrate a modern version of Anscombe's Quartet.

This task will take you through the same journey a data analyst faces: you will start with raw numbers, run the usual summaries, and then discover, through visualisation, that numbers alone were hiding the story.

---

## Part 1: Load and Inspect the Data

Start by importing the libraries you need and loading the dataset.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Configure plot style
sns.set_theme(style='whitegrid', palette='tab10')
plt.rcParams['figure.dpi'] = 100

In [ ]:
# Load the dataset
df = pd.read_csv('datasaurus.csv')

# Preview the first rows
df.head(10)

### 1.1. Structure and data types

Before computing anything, always understand what you are working with.

In [ ]:
# Shape of the dataset (rows, columns)
print('Shape:', df.shape)

# Column names and data types
print('\nDtypes:')
print(df.dtypes)

In [ ]:
# How many unique sub-datasets are there, and how many rows does each contain?
print('Unique datasets:', df['dataset'].nunique())
print('\nRows per dataset:')
print(df['dataset'].value_counts())

### 1.2. Overall summary statistics

Use `describe()` to get a global numerical summary of `x` and `y`.

In [ ]:
# Summary statistics for the entire dataset
df[['x', 'y']].describe().round(2)

---

## Part 2: Grouped Statistics: The Reveal

The dataset column holds 13 different named groups. Let's compute summary statistics **per group** and see if the groups differ.

In [ ]:
# Compute mean and standard deviation of x and y for each sub-dataset
grouped_stats = (
    df.groupby('dataset')[['x', 'y']]
    .agg(['mean', 'std'])
    .round(2)
)

grouped_stats

In [ ]:
# Also compute the Pearson correlation between x and y per group
correlation = df.groupby('dataset').apply(lambda g: g['x'].corr(g['y'])).round(2)
correlation.name = 'corr(x,y)'
print(correlation)

> **Question:** Look at the table above. Are the 13 datasets statistically different from each other?  
> Write your answer in the cell below before moving on.


---

<!-- ## Part 3: Now Let us Actually Look at the Data

We will focus on three sub-datasets: **`dino`**, **`star`**, and **`bullseye`**. These three were chosen because they produce a dramatic visual contrast despite their identical statistics.

Later, feel free to explore the remaining 10 groups. -->

In [ ]:
# Filter to the three focus datasets
focus = ['dino', 'star', 'bullseye']
df_focus = df[df['dataset'].isin(focus)].copy()

print(f'Rows in subset: {len(df_focus)}')

### 3.1 — Individual scatter plots

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharey=True)

colors = sns.color_palette('tab10', 3)

for ax, name, color in zip(axes, focus, colors):
    subset = df_focus[df_focus['dataset'] == name]
    ax.scatter(subset['x'], subset['y'], color=color, alpha=0.7, s=40, edgecolors='white', linewidths=0.4)
    ax.set_title(name, fontsize=14, fontweight='bold')
    ax.set_xlabel('x')
    ax.set_ylabel('y')

fig.suptitle('Same statistics, completely different data', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

### 3.2 — Side-by-side with statistics overlay

Let's add the mean and standard deviation annotations to make the point explicit.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5.5), sharey=True)

for ax, name, color in zip(axes, focus, colors):
    subset = df_focus[df_focus['dataset'] == name]
    
    ax.scatter(subset['x'], subset['y'], color=color, alpha=0.65, s=40,
               edgecolors='white', linewidths=0.4, label='observations')
    
    # Mean crosshair
    mx, my = subset['x'].mean(), subset['y'].mean()
    ax.axvline(mx, color='black', linestyle='--', linewidth=1.0, alpha=0.6)
    ax.axhline(my, color='black', linestyle='--', linewidth=1.0, alpha=0.6)
    ax.scatter([mx], [my], color='black', s=80, zorder=5, label=f'mean ({mx:.1f}, {my:.1f})')
    
    # Stats box
    stats_text = (
        f"mean x = {subset['x'].mean():.2f}\n"
        f"mean y = {subset['y'].mean():.2f}\n"
        f"sd x   = {subset['x'].std():.2f}\n"
        f"sd y   = {subset['y'].std():.2f}\n"
        f"corr   = {subset['x'].corr(subset['y']):.2f}"
    )
    ax.text(0.03, 0.97, stats_text, transform=ax.transAxes,
            fontsize=8.5, verticalalignment='top', fontfamily='monospace',
            bbox=dict(boxstyle='round,pad=0.4', facecolor='white', alpha=0.85, edgecolor='grey'))
    
    ax.set_title(name, fontsize=14, fontweight='bold')
    ax.set_xlabel('x')
    ax.set_ylabel('y')

fig.suptitle('Datasaurus Dozen — statistics are identical, shapes are not',
             fontsize=14, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()

> **❓ Question:** What would a data analyst have concluded if they had only looked at the summary statistics table?  
> What does this tell you about when and why visualisation is necessary?

*(Double-click to write your answer here)*

---

## Part 4 — Small Multiples: All 13 Datasets at Once

Seaborn's `FacetGrid` makes it easy to produce a *small multiples* plot — the same chart type repeated for each group. This is a powerful pattern for comparing distributions across many categories.

In [ ]:
g = sns.FacetGrid(df, col='dataset', col_wrap=5, height=3, aspect=1.0,
                  sharex=False, sharey=False)
g.map(sns.scatterplot, 'x', 'y', alpha=0.6, s=18, color='steelblue', edgecolor='white', linewidth=0.2)
g.set_titles(col_template='{col_name}', size=10)
g.figure.suptitle('All 13 Datasaurus Dozen datasets — identical statistics',
                   fontsize=13, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()

---

## Some Exploration

For each chart type below, run the cell and then answer the key question:

> **Does this chart type reveal the structural differences between datasets, or does it hide them?**

---

### Histograms

Plot the marginal distribution of `x` and `y` separately for each focus dataset.

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 8), sharey=True)

for col_idx, var in enumerate(['x', 'y']):
    for ax, name, color in zip(axes[col_idx], focus, colors):
        subset = df_focus[df_focus['dataset'] == name]
        sns.histplot(subset[var], ax=ax, color=color, bins=15, kde=False)
        ax.set_title(f'{name} — {var}', fontsize=12, fontweight='bold')
        ax.set_xlabel(var)

fig.suptitle('Histograms — marginal distributions of x and y per dataset',
             fontsize=13, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()


> **Answer:** Partially. Histograms show the marginal distribution of one variable at a time, so they reveal that the datasets differ along each axis individually. But they lose all information about the *relationship* between x and y — you cannot see the dinosaur or the star from a histogram alone. They reveal more than summary statistics, but less than a scatterplot.

---

### KDE plots

Overlay density curves for the three focus datasets on the same axis.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for ax, var in zip(axes, ['x', 'y']):
    sns.kdeplot(data=df_focus, x=var, hue='dataset', ax=ax, fill=True, alpha=0.3, linewidth=1.5)
    ax.set_title(f'KDE of {var} — three focus datasets', fontsize=12, fontweight='bold')
    ax.set_xlabel(var)

fig.suptitle('KDE plots — overlaid density curves per dataset',
             fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()


> **Answer:** Same limitation as histograms — KDE plots show the marginal density of one variable at a time. The three curves look somewhat different from each other (especially for y), but you cannot reconstruct the actual shapes from them. The structural difference between dino, star, and bullseye is heavily underrepresented.

---

### Pair plots

Plot all pairwise combinations of variables, coloured by dataset.

In [ ]:
g = sns.pairplot(df_focus, hue='dataset', plot_kws={'alpha': 0.5, 's': 20},
                 diag_kind='kde', height=3.5)
g.figure.suptitle('Pair plot — dino, star, bullseye', fontsize=13,
                   fontweight='bold', y=1.01)
plt.show()


> **Answer:** Yes — the off-diagonal scatter plot (x vs y) fully reveals the structural differences, showing the dinosaur, star, and bullseye shapes clearly. The diagonal KDE plots add the marginal distributions. For a dataset with only two variables the pair plot is essentially a scatter plot with extras, but the pattern scales well to datasets with many variables.

---

### Box plots

Summarise the distribution of `x` and `y` per dataset using box plots.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for ax, var in zip(axes, ['x', 'y']):
    sns.boxplot(data=df_focus, x='dataset', y=var, ax=ax,
                palette='tab10', width=0.5, linewidth=1.2)
    ax.set_title(f'Box plot of {var} per dataset', fontsize=12, fontweight='bold')
    ax.set_xlabel('dataset')
    ax.set_ylabel(var)

fig.suptitle('Box plots — do they reveal the structural differences?',
             fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()


> **Answer:** No — and this is the most important result. The three box plots look nearly identical for both x and y: same median, same IQR, same whiskers. Box plots summarise only five statistics per group (min, Q1, median, Q3, max), so they suffer the same blindspot as the summary statistics table. The dinosaur, star, and bullseye are completely invisible. Some chart types hide structure rather than revealing it — and box plots are a prime example.

---

## Key Takeaways

- Summary statistics (mean, SD, correlation) can be completely identical across datasets with totally different structure
- Visualisation is not a finishing step — it is a **diagnostic step** that must happen early
- Different chart types reveal different aspects: scatterplots show point-level structure, histograms show marginal distributions, box plots summarise spread but can hide shape
- The small multiples pattern (FacetGrid) is a powerful way to compare many groups at a glance

--> In **Task 2**, you will move to a real-world dataset with real problems — and discover that the "hard work" you just did manually can be partially automated.